In [1]:
# Add root directory to path
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
PROJECT_ROOT = os.path.abspath(os.path.join(
                  os.path.abspath('..'),
                  os.pardir)
)
sys.path.append(PROJECT_ROOT)

# Imports
from electronTransportCode.SimOptions import LineSourceSimulation
from electronTransportCode.SimulationDomain import SimulationDomain
from electronTransportCode.MCParticleTracer import AnalogParticleTracer, MCParticleTracer
from electronTransportCode.MCEstimator import DoseEstimator, FluenceEstimator
from electronTransportCode.ProjectUtils import ERE
from electronTransportCode.ParticleModel import SimplifiedEGSnrcElectron
from electronTransportCode.ProjectUtils import E_THRESHOLD
from electronTransportCode.Material import Material
%matplotlib widget

### Set up all objects

In [2]:
# Set up initial conditions
eSource: float = 1.0
SEED: int = 4  # Random number generator seed
lineSourceSim = LineSourceSimulation(minEnergy=0, eSource=eSource, rngSeed=SEED)

# Set up simulation domain
xmin = -1.5; xmax = 1.5
simDomain = SimulationDomain(xmin, xmax, xmin, xmax, 200, 200)

# Set up dose estimator
fluenceEstimator = FluenceEstimator(simDomain=simDomain, Emin=0.0, Emax=eSource, Ebins=20)

# Set up particle
particle = SimplifiedEGSnrcElectron(generator=SEED)  # rng is later overridden by simulation object 

# Set up particle tracer
particleTracer = AnalogParticleTracer(particle=particle, simOptions=lineSourceSim, simDomain=simDomain)

### Run simulation

In [3]:
NB_PARTICLES = 100
particleTracer(nbParticles=NB_PARTICLES, estimator=fluenceEstimator)
print(f'Average amount of events: {particleTracer.averageNbCollisions}')

IndexError: index 20 is out of bounds for axis 0 with size 20